# Publication "Applying Delta Method in A/B Tests Analysis"

In [1]:
import pandas as pd
import numpy as np
from random import randint
from scipy import stats 

In [2]:
click_control = [randint(0,20) for i in range(10000)]
view_control = [randint(1,60) for i in range(10000)]

click_treatment = [randint(0,21) for i in range(10000)]
view_treatment = [randint(1,60) for i in range(10000)]

control = pd.DataFrame({'click':click_control,'view':view_control})
treatment = pd.DataFrame({'click':click_treatment,'view':view_treatment})

In [3]:
control.head()

,click,view
0,5,16
1,10,7
2,7,59
3,7,37
4,17,43


In [4]:
treatment.head()

,click,view
0,13,29
1,8,12
2,4,1
3,17,28
4,17,23


In [5]:
# Variance estimation of metrics ratio
def var_ratio(x,y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x,ddof=1)
    var_y = np.var(y,ddof=1)
    cov_xy = np.cov(x,y,ddof=1)[0][1]
    result = (var_y/mean_y**2 - 2*cov_xy/(mean_x*mean_y) + var_x/mean_x**2) * (mean_x**2)/(mean_y**2) * 1/len(x)
    return result

In [6]:
# ttest calculation 
def ttest(mean_control,mean_treatment,var_control,var_treatment):
    diff = mean_treatment - mean_control
    var = var_control+var_treatment
    stde = 1.96*np.sqrt(var)
    lower = diff - stde 
    upper = diff + stde
    z = diff/np.sqrt(var)
    p_val = stats.norm.sf(abs(z))*2

    result = {'mean_control':mean_control,
             'mean_experiment':mean_treatment,
             'var_control':var_control,
             'var_experiment':var_treatment,
             'difference':diff,
             'lower_bound':lower,
             'upper_bound':upper,
             'z':z,
             'p-value':p_val}
    return pd.DataFrame(result,index=[0])

In [7]:
var_control = var_ratio(control['click'],control['view'])
var_treatment = var_ratio(treatment['click'],treatment['view'])
mean_control = control['click'].sum()/control['view'].sum()
mean_treatment = treatment['click'].sum()/treatment['view'].sum()

In [8]:
ttest(mean_control,mean_treatment,var_control,var_treatment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,z,p-value
0,0.325727,0.341581,0.000007,0.000008,0.015854,0.008141,0.023567,4.028824,0.000056
